In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
my_words = ["select", "from", "where", "group"]
my_words_similar = pd.DataFrame([map(lambda x: x[0], glove_wiki_vectors.similar_by_word(word)) for word in my_words], index=my_words).T

display(my_words_similar)

,select,from,where,group
0,selected,in,.,groups
1,choose,into,now,based
2,chosen,came,there,members
3,selecting,through,when,organization
4,selection,while,once,led
5,choosing,of,it,which
6,pick,after,one,formed
7,options,for,which,alliance
8,decide,the,well,joined
9,hold,back,they,called


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [7]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [8]:
word_pairs_cosine = pd.DataFrame([[glove_wiki_vectors.similarity(x, y) for x, y in word_pairs]], columns=word_pairs).T

display(word_pairs_cosine)

,0
"(coast, shore)",0.700027
"(clothes, closet)",0.546276
"(old, new)",0.643249
"(smart, intelligent)",0.755273
"(dog, cat)",0.879807
"(tree, lawyer)",0.076719


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [9]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [10]:
test_words_presence = pd.DataFrame([[word in glove_wiki_vectors for word in test_words]], columns=test_words).T

display(test_words_presence)

,0
covididiot,False
fomo,False
frenemies,True
anthropause,False
photobomb,False
selfie,False
pxg,False
pacg,False
cct,True
escc,True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [13]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283244

In [14]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [15]:
bias_1_word_pairs = [("straight", "hiv/aids"), ("gay", "hiv/aids")]
bias_1 = pd.DataFrame([[glove_wiki_vectors.similarity(x, y) for x, y in bias_1_word_pairs]], columns=bias_1_word_pairs).T

display(bias_1)

,0
"(straight, hiv/aids)",-0.214362
"(gay, hiv/aids)",0.248810


In [16]:
bias_2_word_pairs = [("jewish", "elite"), ("gentile", "elite")]
bias_2 = pd.DataFrame([[glove_wiki_vectors.similarity(x, y) for x, y in bias_2_word_pairs]], columns=bias_2_word_pairs).T

display(bias_2)

,0
"(jewish, elite)",0.346619
"(gentile, elite)",0.042662


In [17]:
bias_3_word_pairs = [("cis", "suicide"), ("trans", "suicide")]
bias_3 = pd.DataFrame([[glove_wiki_vectors.similarity(x, y) for x, y in bias_3_word_pairs]], columns=bias_3_word_pairs).T

display(bias_3)

,0
"(cis, suicide)",-0.015828
"(trans, suicide)",0.186493


In [18]:
analogy("man", "murderer", "woman")

man : murderer :: woman : ?


,Analogy word,Score
0,rapist,0.753811
1,inmate,0.633734
2,molester,0.622330
3,murderers,0.616305
4,rapists,0.609310
5,accomplice,0.600802
6,prostitute,0.594909
7,killer,0.593983
8,murder,0.589735
9,murdering,0.585470


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

**1.** Yes. The examples above demonstrate that there are still biases present in the embeddings. For example, the vectors for "cis" and "suicide" are orthogonal whereas the vectors for "trans" and "suicide" are somewhat collinear. This reinforces the harmful stereotype that transgender identity is a mental illness or inherently linked with mental health challenges.

**2.** Continuing with the previous example, a classification model trained on these embeddings for content moderation on a social media platform could flag transgender support, outreach, or educational material as inappropriate because of its association with sensitive topics such as suicide. This would lead to the unintentional censorship and stigmatization of transgender individuals.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [19]:
from sklearn.datasets import fetch_20newsgroups

In [20]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [21]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [22]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [23]:
def preprocess_text(text: str, nlp=nlp):
    return " ".join(normalize(token).strip() for token in nlp(text) if not token.is_punct and not token.is_space and not token.is_stop)

def normalize(token):
    return "[url]" if token.like_url else "[num]" if token.like_num else "[email]" if token.like_email else token.norm_

In [24]:
df["text_pp"] = df["text"].apply(preprocess_text)

display(df.head())

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know reading [num] u.s.c. [num] sence wonderin...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question refs list algorithm thinking bit ...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea having newsgroup a...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bugging appreciate help crea...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,latest upi foreign ministry spokesman ferhat a...


In [25]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea having newsgroup a...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bugging appreciate help crea...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,latest upi foreign ministry spokesman ferhat a...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,hi like subscribe leadership magazine wonder d...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

1. Drop punctuation, whitespace, and stopwords. Tokens in this category do not provide any significantly meaningful information about the content of the text,
so including them would add unnecessary noise to the topic modeling process.

2. Replace URLs, email addresses, and numbers with generic tokens. Tokens in this category are often too unique to be useful for prediction, so it's more useful to know that a URL is present in the text than to know the specific URL.

3. Normalize remaining tokens by taking the etymological root of each word. Most of the semantic meaning of a word is contained in the root rather than the conjugation,
so this step reduces the vocabulary size and improves the signal-to-noise ratio.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

**1.** Recall that our dataset was sampled from 8 news categories. Broadly speaking, each of these news categories falls into one of four topics:

| category               | topic    |
| ---------------------- | -------- |
| rec.sport.hockey       | sports   |
| rec.sport.baseball     | sports   |
| soc.religion.christian | religion |
| alt.atheism            | religion |
| comp.graphics          | tech     |
| comp.windows.x         | tech     |
| talk.politics.mideast  | politics |
| talk.politics.guns     | politics |

Therefore, we will choose `n_components=4` for the number of topics. This approach allows us to evaluate the accuracy of our model by examining whether the predicted topics align with the actual categories in the dataset, while keeping the number of topics relatively low to help with interpretation.

In [26]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=4, random_state=42)
count_vectorizer = CountVectorizer()

text_pp = count_vectorizer.fit_transform(df["text_pp"])
topics = lda.fit_transform(text_pp)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

**1.** Based on the top 10 words for each topic, it would be reasonable to assign the following labels to each topic:

| topic | label    |
| ----- | -------- |
| 0     | sports   |
| 1     | politics |
| 2     | religion |
| 3     | tech     |

Overall, the labels seem to match up with the four broad topics we identified in the dataset, which is a good sign. Additionally, the top words for each topic are mostly relevant in terms of semantic meaning, and the distinction between topics is clear. This indicates that the model is performing well in terms of word-topic association.

In [27]:
from mglearn.tools import print_topics

feature_names = count_vectorizer.get_feature_names_out()
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]

print_topics(range(4), feature_names, sorting, n_words=10)

topic 0       topic 1       topic 2       topic 3       
--------      --------      --------      --------      
num           num           num           num           
team          armenian      people        url           
game          turkish       god           email         
year          armenians     think         file          
play          turkey        know          use           
games         turks         like          program       
season        armenia       time          window        
hockey        greek         said          image         
players       genocide      believe       available     
league        people        way           files         




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

**1.** See below.

**2.** Looking at the first five documents, the accuracy of the model seems to be rather low. For example, the first document is assigned to topic 0 (religion) but the actual category is talk.politics.guns (politics). Out of the five documents, three are assigned to the wrong topic this way.

Even though the model performed well on word-topic association, it struggled with document-topic association. This could be due to the fact that the documents are relatively short and contain a limited amount of information, making it difficult for the model to accurately assign them to the correct topic.

In [28]:
suggested_labels = ["sports", "politics", "religion", "tech"]

df["predicted_target"] = topics.argmax(axis=1)
df["predicted_target_name"] = [suggested_labels[topic] for topic in df["predicted_target"]]

display(df.head())

,text,target,target_name,text_pp,predicted_target,predicted_target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know reading [num] u.s.c. [num] sence wonderin...,2,religion
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question refs list algorithm thinking bit ...,3,tech
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea having newsgroup a...,2,religion
3,"The following problem is really bugging me,\na...",2,comp.windows.x,following problem bugging appreciate help crea...,1,politics
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,latest upi foreign ministry spokesman ferhat a...,1,politics


<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

**1.** Content-based filtering uses item features to make recommendations. It works by training a model for each user, which associates ratings with the features of items the user has previously interacted with. For example, if a user tends to rate cheaper restaurants more highly, the model might learn an inverse relationship between price and rating. When a new item is introduced, the model can predict the user's preference for that item based on its features and the system can recommend items with the highest predicted ratings.

This approach is useful when features are available for each item and there is a relationship between those features and the user's preferences, but can struggle otherwise.

**2.** One consequence of using recommender systems is the potential for filter bubbles, where users are only exposed to content that aligns with their current preferences. This can limit the diversity of the content that is recommended to users, which can reinforce biases and limit exposure to new ideas.

Another consequence is the potential for privacy violations. Recommender systems rely on user data to make recommendations, which can lead to concerns about data security and the misuse of personal information. Users may be uncomfortable with the data collection required to make accurate recommendations.

**3.** Transfer learning involves training a model on a large dataset for a certain task, then fine-tuning the model on a smaller dataset for a different related task. This approach leverages knowledge from the first task to improve performance on the second task, even if the second task has a limited amount of training data.

In practice, one example of transfer learning is when using a pre-trained model such as Word2Vec or GloVe in a downstream task like sentiment analysis or topic modelling for a smaller dataset. The pre-trained model has already learned the relationships between words from a large corpus, so it can be fine-tuned on the smaller dataset to produce better results (such as embeddings) than training from scratch.

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)